## Social Media Monitoring and Analysis

Welcome! This notebook is geared to give you starter examples of how to use ChatGPT for social media monitoring and analysis. Specifically we will look at three use cases:
- Sentiment and Emotion Analysis
- Topic Extraction
- Classification / Categorization

For our purposes we will assume we work at Sam's Club (a well-known retail warehouse chain owned by Wal-Mart) and have been asked to look at a series of tweets for our analysis. I'll walk you through all the basic steps needed to do the analysis. 

NOTE: To make it easier to focus on the process of analysis instead of acquiring the data, I've included a JSON file that has curated output from Twitter (now called, X) in it. If you want to sign up for a developer account so you can get live data, go here: https://developer.twitter.com/

### Sentiment and Emotion Analysis

The first order of business is to take the data and put it into a format that is easier to analyze. Take a moment to look at the Tweets.json file and see how we get information from Twitter. Notice that the file is a list of dictionaries where each dictionary represents a tweet. We need to manipulate it into something usable. Let's begin!

In [39]:
# import our packages 
import pandas as pd
import json
import os
import openai

# set our key for openai to be used later
openai_key = os.getenv('OPENAI_KEY')
openai.api_key = openai_key


### Read the Data

Now that we have our packages ready to go and our OpenAI API key set we need to read the data info a pandas dataframe to ease our cleaning and analysis efforts.

In [40]:
# Start by inspecting the json file and determining the structure of the data. Remember that the file is a list of dictionaries where each dictionary represents a tweet. 
# Some are nested dictionaries and some are not. We need to flatten the data so that we can convert it into a DataFrame.
# The json_normalize function from pandas is used to convert JSON data into a flat table (DataFrame).
from pandas import json_normalize

# Open the file named "Tweets.json" for reading. The "with" statement ensures that the file is properly closed after it is no longer needed.
with open("Tweets.json") as file:
    # Use the json.load function to load the JSON data from the file into a Python object (usually a list or a dictionary).
    data = json.load(file)

# Convert the JSON data into a DataFrame. json_normalize flattens the data, meaning it can create a DataFrame from nested JSON data.
df = json_normalize(data)

# Print the first 5 rows of the DataFrame to see what the data looks like.
print(df.head())

# Save the DataFrame to a CSV file named "initaldataframe.csv" so you can see the progress. The argument index=False means that the DataFrame's index will not be saved in the CSV file.
# While this step isn't necessary, it's a good idea to save your work as you go along and to check that the data looks correct.
# If you are using VS Code, I highly recommend installing the Excel Viewer extension or Rainbow CSV extension so you can view the CSV file more easily. Or just open the file in Excel as well.
df.to_csv('initaldataframe.csv', index=False)


                       created_at                   id               id_str  \
0  Mon May 15 18:21:47 +0000 2023  1658175862045302797  1658175862045302797   
1  Mon May 15 17:29:18 +0000 2023  1658162654479888397  1658162654479888397   
2  Mon May 15 17:04:13 +0000 2023  1658156342199332864  1658156342199332864   
3  Mon May 15 16:54:52 +0000 2023  1658153988703809536  1658153988703809536   
4  Mon May 15 16:54:24 +0000 2023  1658153870575521793  1658153870575521793   

                                           full_text  truncated  \
0  KitchenAid Mixer Sale on @SamsClub | $90 Off P...      False   
1           @KellieShai_ All we need is some milk. 🍪      False   
2  Happy National Chocolate Chip Day! No denying ...      False   
3  This is heartbreaking that less than a year im...      False   
4  @VIZIOsupport I bought a vizio tv from @SamsCl...      False   

  display_text_range                                             source  \
0           [0, 128]  <a href="https://mobile.t

### Cleaning the Data Up

We will do some very simple cleanup to make dealing with our data easier. First, let's take out columns that we obviously won't need for our goals to make the data even more easy to analyze. Take a look at the initaldataframe.csv and determine what columns you think should be kept for our analysis. At this early stage it's usually a good idea to keep a column if in doubt. We can always trim it out later. Since we are keeping it simple we can hack and slash a bit more than usual. 


In [41]:
# Get a list of all column names
all_columns = df.columns.tolist()

# Print the list of column names
print(all_columns)


['created_at', 'id', 'id_str', 'full_text', 'truncated', 'display_text_range', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'possibly_sensitive', 'lang', 'entities.hashtags', 'entities.symbols', 'entities.user_mentions', 'entities.urls', 'metadata.iso_language_code', 'metadata.result_type', 'user.id', 'user.id_str', 'user.name', 'user.screen_name', 'user.location', 'user.description', 'user.url', 'user.entities.url.urls', 'user.entities.description.urls', 'user.protected', 'user.followers_count', 'user.friends_count', 'user.listed_count', 'user.created_at', 'user.favourites_count', 'user.utc_offset', 'user.time_zone', 'user.geo_enabled', 'user.verified', 'user.statuses_count', 'user.lang', 'user.contributors_enabled', 'user.is_translator', 'user.is_translation_enable

In [42]:
# Define the base column names you want to keep
# copy and paste from the output above to make sure you get the column names correct
base_columns_to_keep = ['created_at', 'id', 'full_text', 'in_reply_to_screen_name']

# Add the metadata and user columns to the list since those are somewhat interesting to us
columns_to_keep = base_columns_to_keep + [col for col in all_columns if col.startswith('metadata.') or col.startswith('user.')]

# Keep only the desired columns in the DataFrame
df = df[columns_to_keep]

# print out the results to a csv file to check them
df.to_csv('limitedcolumns.csv', index=False)

In [43]:
# Looking at limitedcolumns.csv, it looks like we can safely remove some columns
# Let's remove the columns that have no data in them
df = df.dropna(axis=1, how='all')

# print out the results to a csv file to check them
df.to_csv('limitedcolumns_2.csv', index=False)

In [44]:
# Now let's remove the columns that have the word "url" or "color" in them since they don't seem to be useful
df = df[df.columns.drop(list(df.filter(regex='url|color')))]

# print out the results to a csv file to check them
df.to_csv('limitedcolumns_3.csv', index=False)

In [45]:
# drop the withheld_in_countries column because it doesn't have usable values
df.drop(columns=['user.withheld_in_countries'], inplace=True)

# let's also change all the columns with boolean values to be 1 or 0 instead of True or False
# this makes it easier to work with the data later
df = df.replace({True: 1, False: 0})

# we aren't interested in tweets from sams's club and there is another account called SamsClub_Sam that we want to get rid of
# since we have a pretty good size dataset for our purposes let's just get rid of any row that has a username with the words "SamsClub" in it regardless of case
# Remove rows where 'user.screen_name' or 'user.name' contains 'samsclub'
df = df[~(df['user.screen_name'].str.contains('samsclub', case=False) | df['user.name'].str.contains('samsclub', case=False))]


# print out the results to a csv file to check them
df.to_csv('limitedcolumns_4.csv', index=False)

In [46]:
# we have trimmed the data down to 27 columns now and the data looks much more manageable
# a couple of more cleanup items and we will be ready to start analyzing the data
# let's convert all the column names to lowercase
df.columns = df.columns.str.lower()

# finally, let's replace any "." in the column names with "_" to be consistent
df.columns = df.columns.str.replace('.', '_')

# print out the results to a csv file to check them
df.to_csv('limitedcolumns_5.csv', index=False)

### Sentiment and Emotion Analysis
Now on to the first set of analysis: 
For each Tweet, we need to find out the main sentiment (Positive, Neutral, Negative) and the main emotion (Joy, Surprise, Neutral, Sadness, Mistrust, and Disgust)
We will want to return, both, the predicted sentiment and emotion, as well as the score (ranging for -1 (negative) to 1 (positive) for Sentiment and 0 (disgust) to 1 (joy) for emotion).


In [47]:
# define a function to take in our dataframe and return a new dataframe 
# with the columns for sentiment and emotion added to each tweet
def sentiment_emotion_analysis(df):
    # Initialize empty lists to hold sentiment and emotion data
    sentiments = []
    emotions = []

    # Loop through every tweet in the dataframe's 'full_text' column using Few-Shot Prompting
    for tweet in df['full_text']:
        # Define the prompt to be used for Few-Shot Prompting
        # Here is more information on Few-Shot Prompting: https://help.openai.com/en/articles/6654000-best-practices-for-prompt-engineering-with-openai-api 
        prompt=f"""
        Analyze the sentiment of the following tweet and provide a sentiment rating from -1 (completely negative) to 1 (completely positive) and all values in between. If the sentiment is neutral, provide a score of 0. Also, provide an emotion associated with the tweet, choosing from Joy, Surprise, Neutral, Sadness, Mistrust, and Disgust respectively. Rate the emotions from complete joy(1) down to complete disgust(0), assigning values to the emotions as follows: Joy = 1, Surprise = 0.80, Neutral = 0.60, Sadness = 0.40, Mistrust = 0.20, and Disgust = 0. 

        Here are some examples:

        Tweet: Who is making these decisions @SamsClub Do you hate your employees?? Are you kidding?! The @weatherchannel is here. Put it on tv, pay attention to what they are saying, and tell your employees to stay home! #HurricaneIdalia
        Sentiment: -0.8
        Emotion: Disgust

        Tweet: Sams club has frozen grill cheese sandwiches 🤔🤔sheer genius 😍😍
        Sentiment: 0.8
        Emotion: Joy

        Tweet: signed up for a sams club membership. this feel wayyy too grown for me
        Sentiment: -0.2
        Emotion: Sadness

        Tweet: I really need to go sams club
        sentiment: 0.0
        Emotion: Neutral

        Tweet: moms coming home@with sams club pizza!!
        Sentiment: 0.9
        Emotion: Surprise

        Provide the sentiment and emotion ratings in the following JSON format:
        {{"Sentiment": "<sentiment>", "Sentiment_Value": <sentiment_value>, "Emotion": "<emotion>", "Emotion_Value": <emotion_value>}}

        What is the sentiment and emotion of this tweet:
        "{tweet}"

        Answer: {{"Sentiment": "<sentiment>", "Sentiment_Value": <sentiment_value>, "Emotion": "<emotion>", "Emotion_Value": <emotion_value>}}
        
        """

        # Create an API call to OpenAI using the specified model, prompt, and parameters
        # We can play with the parameters to see if we can get better results
        response = openai.Completion.create(
            # This is the model we want to use to generate the content - in this case we are using a ChatGPT 3.5 model because it is cheaper to use than the GPT-4 model
            # Useful for prototyping and testing then using the GPT-4 model for later iterations
            # At the time of this writing, for input the GPT-3.5 model costs $0.0015 per 1000 tokens and the GPT-4 model costs $0.03 per 1000 tokens; for output the 
            # GPT-3.5 model costs $0.002 per 1000 tokens and the GPT-4 model costs $0.06 per 1000 tokens
            # Quite a cost differential between the two models so start cheap to prototype and test and then use the GPT-4 model for later on
            model="text-davinci-003",  

            # This is the prompt we created above
            prompt=prompt,  

            # This is the temperature parameter - it controls the randomness of the output or, put another way, how "creative" the AI is
            # The number can be between 0 and 1 - the closer to 0 the less "creative" the AI is and as the number approaches 1 the more "creative" the AI is
            # Since we are doing an objective analysis we want the AI to be less creative so we set the temperature to 0.2
            # If we were creating children's stories, for example, we would want the AI to be more creative so we set the temperature to 0.7
            temperature=0.2,  

            # This is the max_tokens parameter - The maximum number of tokens to generate in the completion. 
            # The token count of your prompt plus max_tokens cannot exceed the model’s context length. 
            # Most models have a context length of 2048 tokens (except for the newest models, which support 4096).
            # To quote OpenAI: "You can think of tokens as pieces of words, where 1,000 tokens is about 750 words."
            # Our output is fairly small per try so we set the max_tokens to 200 to give the AI enough room to generate the content
            # But small enough to keep the cost down or, at least, error out so can know the limit we set is too small
            max_tokens=2000,  

            # This is the top_p parameter - it controls the range of words chosen for the output
            # The values are from 0 to 1 - the closer to 0 the less diverse the output and the closer to 1 the more diverse the output
            # For example, if we set top_p to 0.5 then the AI will only use the top 50% of the most likely words
            # But, if we set top_p to 1.0 then the AI will use all of the words
            # We are doing an objective analysis so we want the AI to be less diverse so we set the top_p to 0.5
            # If we were creating children's stories we would want the AI to be more diverse so we set the top_p to 1.0
            top_p=0.5,  

            # This is the frequency_penalty parameter - this parameter is used to discourage the model from repeating the same words or phrases too frequently within the generated text.
            # A higher frequency_penalty value will result in the model being more conservative in its use of repeated words. 
            # The values are from -2.0 to 2.0 - the closer to -2.0 the more likely the AI will repeat the same words and the closer to 2.0 the less likely the AI will repeat the same words
            # Typical setting for this parameter is 0.0 or to 1 for eliminating repetition in output.
            frequency_penalty=0.0,  

            # This is the presence_penalty parameter - this parameter is used to encourage the model to include a diverse range of words in the generated text. 
            # A higher presence_penalty value will result in the model being more likely to generate words that have not yet been included in the generated text.
            # The values are from -2.0 to 2.0 - the closer to -2.0 the more likely the AI will repeat the same words and the closer to 2.0 the more likely to include words not used before
            # As with the frequency_penalty parameter, typical setting for this parameter is 0.0 or to 1 for eliminating repetition in output. 
            presence_penalty=0.0 
        )

        # The hard part is done - now we just need to process the response from the API
        # Get the response text and remove the text "Answer: " from the beginning of the response
        response_text = response['choices'][0]['text'].strip().replace("Answer: ", "")

        # Process the response text
        try:
            # Load the response into a JSON object
            response_json = json.loads(response_text)

            # Extract the sentiment and emotion data from the JSON object
            sentiment = response_json["Sentiment"]
            sentiment_value = response_json["Sentiment_Value"]
            emotion = response_json["Emotion"]
            emotion_value = response_json["Emotion_Value"]

            # Append the sentiment and emotion data to the corresponding lists we created earlier
            sentiments.append((sentiment, sentiment_value))
            emotions.append((emotion, emotion_value))

        # If there's an error processing the response text, append "Error" and 0 to the data
        except ValueError as e:
            print(f"Error processing tweet: {tweet}")
            print(f"Response from API: {response_text}")
            sentiments.append(("Error", 0))
            emotions.append(("Error", 0))

    # Add the sentiment and emotion data to the dataframe as new columns
    df['sentiment'], df['sentiment_value'] = zip(*sentiments)
    df['emotion'], df['emotion_value'] = zip(*emotions)

    # Return the updated dataframe
    return df

# Call the function and update the dataframe
df = sentiment_emotion_analysis(df)

# Save the updated dataframe as a CSV file to check the results
df.to_csv('SentimentEmotion.csv', index=False)


### Topic Extraction
To enhance our data for analysis, the business has asked for each Tweet to be assigned a topic. This will aid in analysis later on and make it easier to work with segments of the data.


In [49]:
# Define a function named get_topic that takes a tweet text as an input and determines the topic
def get_topic(text):
    # work up our prompt to be used for Few-Shot Prompting
    prompt = f"""
    Given the following tweet, determine the topic or subject in one or two words. The topic should be broad enough to cover a large number of tweets, but specific enough to be useful. Don't include any punctuation at the end of the topic.

    Here are some examples:

    Tweet: Sams club scan & go for the gas station and the store is the best shit ever.
    The topic of this tweet is: Scan & Go

    Tweet: I didn’t know I needed to invest in bath towels until I got some new ones from sams club 🤩
    The topic of this tweet is: Bath Towels

    Tweet: Disneyland animatronic dragons: 0, Sam’s Club animatronic dragons: 1
    Topic: Inflatables / Airblown

    Tweet: Sam’s Club’s $1.38 hot dog combo powers it past Costco in sales war 
    Topic: Cafe Food

    Tweet: @RateYourCharge but your charge update here at Sam's club in Salt Lake City. We tried to charge on charger 7 plugged in a failed to initialize. Now I plugged in to charge your five and sells welcome Joy which is not me which is kind of weird but it seems to be charging now.
    Topic: EV Charging

    Tweet: "{text}"

    The topic of this tweet is:"""
    
    # Create an API call to OpenAI using the specified model, prompt, and parameters
    # We can play with the parameters to see if we can get better results
    response = openai.Completion.create(
        model="text-davinci-003",  
        prompt=prompt,  
        temperature=0.6, 
        max_tokens=200,  
        top_p=1.0,  
        frequency_penalty=0, 
        presence_penalty=0  
        
    )
     # The model's response is processed to extract the text, remove unnecessary strings and strip leading/trailing white spaces.
    response_text = response['choices'][0]['text'].strip().replace("Answer: ", "")
    
    # The processed response text, which is the predicted topic, is returned from the function.
    return response_text

# The get_topic function is applied to the 'full_text' column and the results are stored in a new column 'topic'.
df['topic'] = df['full_text'].apply(get_topic)

# The training dataframe with the new 'topic' column is saved to a CSV file 
df.to_csv('TopicExtraction.csv', index=False)

### Classification / Categorization

To further enhance the data the analytics department wants us to categorize each Tweet based on it's content into one of the following categories: Content Quality, Customer Support, Spam, Membership issues, Marketing and Other. For each Tweet, we should calculate the category and a score associated tho the category that shows the intensity of the category chosen. 

In [50]:
# define a function to take in our tweet and categorize it
def categorize_tweet(text):
    # A Few-Shot prompt is defined which includes the tweet and instructions for the model to categorize the tweet.
    prompt = f"""
    Given the following tweet, please categorize it into one of the following categories with a score from 0 (not relevant) to 1 (highly relevant) and all values in between based on how relevant the tweet is in relation to the category. 

    Examples:
    
    Tweet: Finna go to sams club and get a box of nature valley bars
    Category: Other
    Score: 0.5

    Tweet: 🚨 BRAND NEW JUNE @SamsClub 2023 VIDEO 👉🏼
    Category: Marketing
    Score: 1

    Tweet: Sam’s Club membership expires soon. May join Costco for a year to compare. 🤔
    Category: Membership issues
    Score: 0.8

    Tweet: Person’s complaint is on spam emails from ADT, along w Sam’s Club, Chase Bank, Wells Fargo, etc. sent to his inbox. Also mentions a hack to ADT’s systems.
    Category: Spam
    Score: 0.9

    Tweet: I sent a Email Tuesday to customer support  because I did not see the Add an offer code' visible in the Sam’s club site at check out I haven’t received a reply yet
    Category: Customer Support
    Score: 0.7

    Tweet: Sam's Club sent me an email notice of recall for a readily consumed product. The contaminants both likely lethal, including Clostridium botulinum.  BOTULISM! 
    Category: Content Quality
    Score: 1


    Please provide the category and score in the following JSON format:
    {{ "Category": "<category_name>", "Score": "<score>" }}
    
    The categories are: 

    - Content Quality
    - Customer Support
    - Spam
    - Membership issues
    - Marketing
    - Other

    Tweet: "{text}"

    Answer: <json output>
    """
    
    # Create an API call to OpenAI using the specified model, prompt, and parameters
    # We can play with the parameters to see if we can get better results
    response = openai.Completion.create(
        model="text-davinci-003",  
        prompt=prompt,  
        temperature=0.2,  
        max_tokens=2000,  
        top_p=0.5,  
        frequency_penalty=0.0,  
        presence_penalty=0.0  
    )
    
    # process the response to extract the text and remove unnecessary text
    response_text = response['choices'][0]['text'].strip().replace("Answer: ", "")

     # convert the response to a Python dictionary using json.loads
    result_dict = json.loads(response_text)
    
    # The category and score are extracted from the dictionary and returned from the function
    category = result_dict["Category"]
    score = float(result_dict["Score"])
    return category, score

# The categorize_tweet function is applied to the 'full_text' column of the dataframe. The returned category and score are stored in two new columns in the dataframe.
# By calling zip(*), we unpack the data into two separate series, which are then separately assigned to the new DataFrame columns 'category' and 'category_score'. 
df['category'], df['category_score'] = zip(*df['full_text'].apply(categorize_tweet))

# The updated dataframe, with the new 'category' and 'category_score' columns, is saved to a CSV file
df.to_csv('Classification.csv', index=False)


### Final Result

Here we may decide to trim out a few more columns or clean up the output a little more but, for the most part, we are done with this basic overview. I'll make one more change to a column name that is bugging me and call it good for now.

In [51]:
# replace the column name 'id' with 'tweet_id'
df = df.rename(columns={'id': 'tweet_id'})


# Now print the columns to check the change and see if any other column names are bugging us
print(df.columns)

# save the dataframe to a CSV file
df.to_csv('FinalResult.csv', index=False)

Index(['created_at', 'tweet_id', 'full_text', 'in_reply_to_screen_name',
       'metadata_iso_language_code', 'metadata_result_type', 'user_id',
       'user_id_str', 'user_name', 'user_screen_name', 'user_location',
       'user_description', 'user_protected', 'user_followers_count',
       'user_friends_count', 'user_listed_count', 'user_created_at',
       'user_favourites_count', 'user_geo_enabled', 'user_verified',
       'user_statuses_count', 'user_contributors_enabled',
       'user_is_translator', 'user_is_translation_enabled',
       'user_profile_background_tile', 'user_profile_use_background_image',
       'user_has_extended_profile', 'user_default_profile',
       'user_default_profile_image', 'user_following',
       'user_follow_request_sent', 'user_notifications',
       'user_translator_type', 'sentiment', 'sentiment_value', 'emotion',
       'emotion_value', 'topic', 'category', 'category_score'],
      dtype='object')
